In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('compiled_features_dataset.csv')
df.shape

In [ ]:
df.info()

In [ ]:
source_file_list = []
for file in df['path']:
    file = file.split('\\')[-1]
    source_file_list.append(file)

In [ ]:
df['source_file'] = source_file_list

In [ ]:
merge_df = pd.read_csv('reference_dataset.csv')
merge_df.shape

In [ ]:
merge_df = merge_df[['face_index', 'source_file', 'pixels']]

In [ ]:
#added 'source_file' and 'pixels'
df = df.merge(merge_df, left_on=['source_file', 'face_index'], right_on=['source_file', 'face_index'], how='left', validate='one_to_one')
df

In [ ]:
# df['pixels'].isna().sum()

In [ ]:
# for column_name in df.columns:
#     print(column_name)

In [ ]:
def rescale_bbox(current_dim, orig_w, orig_h, x1, y1, x2, y2):
    # The amount of padding that was added, check which padding was used
    pad_x = max(orig_h - orig_w, 0) * (current_dim / max(orig_h, orig_w)) #x was padded if h > w
    pad_y = max(orig_w - orig_h, 0) * (current_dim / max(orig_h, orig_w))

    # (current_dim / max(orig_h, orig_w)) scqles the padding
    # if orig > curr_dim (<1, i.e., become smaller) if curr_dim > oring (>1, larger)
    
    # Image height and width after padding is removed
    unpad_h = current_dim - pad_y
    unpad_w = current_dim - pad_x
    
    print('box scale', orig_w / unpad_w, orig_h / unpad_h)
    print('box', x2-x1 + 1, y2-y1 + 1)
    
#     # Rescale bounding boxes to dimension of original image
    x1 = ((x1 - pad_x // 2) / unpad_w) * orig_w
    y1 = ((y1 - pad_y // 2) / unpad_h) * orig_h
    x2 = ((x2 - pad_x // 2) / unpad_w) * orig_w
    y2 = ((y2 - pad_y // 2) / unpad_h) * orig_h
    
    #     ((x - pad_x // 2) / unpad_w) * orig_w
    #x - padx // 2 removes the padding on left and right
    # /unpad scales (normalize) and then scale up using orig(w)
    
    return x1, y1, x2, y2

def rescale_area(current_dim, orig_w, orig_h, w, h):
    print('wh', w, h, w/h)
    
    pad_x = max(orig_h - orig_w, 0) * (current_dim / max(orig_h, orig_w))
    pad_y = max(orig_w - orig_h, 0) * (current_dim / max(orig_h, orig_w))
    
    # Image height and width after padding is removed
    unpad_h = current_dim - pad_y
    unpad_w = current_dim - pad_x
    
    print('unpad', unpad_w, unpad_h)
    print('scale', orig_w / unpad_w, orig_h / unpad_h)
    
    new_w = w / unpad_w * orig_w
    new_h = h / unpad_h * orig_h
    
    print('new wh', new_w, new_h, new_w/new_h)
    return new_w * new_h

def rescale_area(current_dim, orig_w, orig_h, w, h):
    print('wh', w, h, w/h)
    
    pad_x = max(orig_h - orig_w, 0) * (current_dim / max(orig_h, orig_w))
    pad_y = max(orig_w - orig_h, 0) * (current_dim / max(orig_h, orig_w))
    
    # Image height and width after padding is removed
    unpad_h = current_dim - pad_y
    unpad_w = current_dim - pad_x
    
    print('unpad', unpad_w, unpad_h)
    print('scale', orig_w / unpad_w, orig_h / unpad_h)
    
    new_w = w / unpad_w * orig_w
    new_h = h / unpad_h * orig_h
    
    print('new wh', new_w, new_h, new_w/new_h)
    return new_w * new_h

def rescale_pixels(current_dim, orig_w, orig_h, area):
    pad_x = max(orig_h - orig_w, 0) * (current_dim / max(orig_h, orig_w))
    pad_y = max(orig_w - orig_h, 0) * (current_dim / max(orig_h, orig_w))
    
    # Image height and width after padding is removed
    unpad_h = current_dim - pad_y
    unpad_w = current_dim - pad_x
    
    print('unpad', unpad_w, unpad_h)
    print('scale', orig_w / unpad_w, orig_h / unpad_h)\
    
    print('area', area, (orig_w / unpad_w) ** 2, area * (orig_w / unpad_w) ** 2)
    
    return area * (orig_w / unpad_w) ** 2

# def rescale_pixel(box_w, box_h, x1, y1, x2, y2, pixels, source_w, source_h):
#     current_dim = 128
#     print('box', box_w, box_h, box_w / box_h)
#     #size 128
#     #orig w h - size of the padded bbox
#     pad_x = max(box_h - box_w, 0) * (current_dim / max(box_h, box_w)) #x was padded if h > w
#     pad_y = max(box_w - box_h, 0) * (current_dim / max(box_h, box_w))

#     # Image height and width after padding is removed (current size)
#     unpad_h = current_dim - pad_y
#     unpad_w = current_dim - pad_x
#     print('unpad', unpad_w, unpad_h)
    
#     pixel_w = box_w * ((pixels / (box_w * box_h)) ** 0.5)
#     pixel_h = box_h * ((pixels / (box_w * box_h)) ** 0.5)
    
#     print('pixel', pixel_w, pixel_h, pixel_w / pixel_h, pixel_w * pixel_h)
    
#     print('scale', box_w / unpad_w, box_h / unpad_h)
    
# #     # Rescale bounding boxes to dimension of original image
#     w = pixel_w / unpad_w * box_w
#     h = pixel_h / unpad_h * box_h
    
#     print('wh', w, h, w/h)
    
#     #w x h is the scaled number of pixels with respect to bbox and 416
    
#     current_dim = 416
#     print('orig', orig_w, orig_h, orig_w / orig_h)
#     pad_x = max(orig_h - orig_w, 0) * (current_dim / max(orig_h, orig_w)) #x was padded if h > w
#     pad_y = max(orig_w - orig_h, 0) * (current_dim / max(orig_h, orig_w))
    
#     unpad_h = current_dim - pad_y
#     unpad_w = current_dim - pad_x
#     print('unpad', unpad_w, unpad_h)
    
#     print('scale', orig_w / unpad_w, orig_h / unpad_h)
    
#     w = w / unpad_w * orig_w
#     h = h / unpad_h * orig_h
    
#     print('wh', w, h, w/h)
    
#     return w * h

In [ ]:
scaled_x1_list = []
scaled_y1_list = []
scaled_x2_list = []
scaled_y2_list = []

bbox_area_list = []
scaled_bbox_area_list = []

lbbox_area_list = []
scaled_lbbox_area_list = []

scaled_pixels_list = []

bg_pixels_list = []
scaled_bg_pixels_list = []
count = 0
for index, row in df.iterrows():
    print(row['path'])
    print(row['face_index'])
    orig_h, orig_w = row['source_w'], row['source_h']

    scale_x1, scale_y1, scale_x2, scale_y2 = rescale_bbox(416, orig_w, orig_h, row['x1'], row['y1'], row['x2'], row['y2'])
    scaled_x1_list.append(scale_x1)
    scaled_y1_list.append(scale_y1)
    scaled_x2_list.append(scale_x2)
    scaled_y2_list.append(scale_y2)

    x, y, w, h = (row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2, row['x2'] - row['x1'] + 1, row['y2'] - row['y1'] + 1           

    bbox_area = w * h
    bbox_area_list.append(bbox_area)

    scaled_bbox_area = rescale_area(416, orig_w, orig_h, w, h)
    scaled_bbox_area_list.append(scaled_bbox_area)

    pad_x1 = max(int(np.floor((x - w))), 0) # prevent negative values
    pad_y1 = max(int(np.floor((y - h))), 0)
    pad_x2 = min(int(np.ceil((x + w))), 415) # prevent from getting out of range
    pad_y2 = min(int(np.ceil((y + h))), 415)

    lbbox_area = (pad_x2 - pad_x1 + 1) * (pad_y2 - pad_y1 + 1)
    lbbox_area_list.append(lbbox_area)

    scaled_lbbox_area = rescale_area(416, orig_w, orig_h, pad_x2 - pad_x1 + 1, pad_y2 - pad_y1 + 1)
    scaled_lbbox_area_list.append(scaled_lbbox_area)
    
    scaled_pixels = rescale_pixels(416, orig_w, orig_h, row['pixels'])
    scaled_pixels_list.append(scaled_pixels)
    
    bg_pixels = w * h - row['pixels']
    if (bg_pixels < 0):
        bg_pixels = 0
    
    bg_pixels_list.append(bg_pixels)
    
    scaled_bg_pixels = rescale_pixels(416, orig_w, orig_h, bg_pixels)
    scaled_bg_pixels_list.append(scaled_bg_pixels)
    
        
        


    
#     pad_x, pad_y, pad_w, pad_h = (pad_x1 + pad_x2) / 2, (pad_y1 + pad_y2) / 2, pad_x2 - pad_x1, pad_y2 - pad_y1        
        
#         test = rescale_pixel(pad_w, pad_h, pad_x1, pad_y1, pad_x2, pad_y2, 200, orig_w, orig_h)
        
#         image = cv2.imread(row['path'])
#         image = cv2.rectangle(image, (int(scale_x1), int(scale_y1)), (int(scale_x2), int(scale_y2)), (255, 0, 0), 3) 
#         plt.imshow(image)
#         plt.show()
print(count)

In [ ]:
df['scaled_pixels'] = scaled_pixels_list

df['bg_pixels'] = bg_pixels_list
df['scaled_bg_pixels'] = scaled_bg_pixels_list

df['scaled_x1'] = scaled_x1_list
df['scaled_y1'] = scaled_y1_list
df['scaled_x2'] = scaled_x2_list
df['scaled_y2'] = scaled_y2_list 

df['bbox_area'] = bbox_area_list 
df['scaled_bbox_area'] = scaled_bbox_area_list

df['lbbox_area'] = lbbox_area_list
df['scaled_lbbox_area'] = scaled_lbbox_area_list


In [ ]:
df. rename(columns = {'source_w':'updated_source_h', 'source_h':'updated_source_w'}, inplace = True)

In [ ]:
df.to_csv('recompiled_features_dataset.csv', index=False)